In [1]:
%%capture
!pip install yfinance

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

In [29]:
# ALGORITHM OUTLINE
# (1) Find current_price for stock of interest (use stocks most recent close price)
# (2) Check if .93*current_price \in [low,high] for stock of interest for the last week (last five days)
# algorithm for single date
# dat = yf.Ticker("^TASI.SR")
# history_for_lastweek_5d = dat.history(period='5d')
# low = history_for_lastweek_5d["Low"].min()
# high = history_for_lastweek_5d["High"].max()
# current_price = history_for_lastweek_5d["Close"][0]
# price_of_interest = .93*current_price
# price_of_interest_within_last_week = low <= price_of_interest and price_of_interest <= high
# https://finance.yahoo.com/quote/%5ETASI.SR/

In [39]:
ticker = "^TASI.SR"
data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
#data.tail(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
data.head(3)

# exit and sell stock everytime a one percent loss is realized or exit and sell stock everytime a one percent gain is realized
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)
print("\n\n"+str(data["closeaboveup1%"].sum()))
print("\n"+str(data["closebelowdown1%"].sum()))

# Saudi Telecom Company (7010.SR)
ticker = "7010.SR"
data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
#data.tail(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)
print("\n\n"+str(data["closeaboveup1%"].sum()))
print("\n"+str(data["closebelowdown1%"].sum()))

# Al Rajhi Banking and Investment Corporation (1120.SR)
ticker = "1120.SR"
data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
#data.tail(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)
print("\n\n"+str(data["closeaboveup1%"].sum()))
print("\n"+str(data["closebelowdown1%"].sum()))

# Saudi Basic Industries Corporation (2010.SR)
ticker = "2010.SR"
data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
#data.tail(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)
print("\n\n"+str(data["closeaboveup1%"].sum()))
print("\n"+str(data["closebelowdown1%"].sum()))

# Alinma Bank (1150.SR)
ticker = "1150.SR"
data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
#data.tail(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)
print("\n\n"+str(data["closeaboveup1%"].sum()))
print("\n"+str(data["closebelowdown1%"].sum()))

# Jarir Marketing Company (4190.SR)
ticker = "4190.SR"
data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
#data.tail(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
#data.head(3)
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)
print("\n\n"+str(data["closeaboveup1%"].sum()))
print("\n"+str(data["closebelowdown1%"].sum()))

[*********************100%***********************]  1 of 1 completed




8

18


[*********************100%***********************]  1 of 1 completed




10

47


[*********************100%***********************]  1 of 1 completed


...



5

52


[*********************100%***********************]  1 of 1 completed




12

79


[*********************100%***********************]  1 of 1 completed




13

55


In [38]:
saudi_tickers = [1010,1020,1030,1050,1060,1080,1111,1120,1140,1150,1180,1182,1183,1201,1202,1210,1211,1212,1213,1214,1301,1302,1303,1304,1320,1321,1322,1810,1820,1830,1831,1832,1833,1834,1835,2001,2010,2020,2030,2040,2050,2060,2070,2080,2081,2082,2083,2084,2090,2100,2110,2120,2130,2140,2150,2160,2170,2180,2190,2200,2210,2220,2222,2223,2230,2240,2250,2270,2280,2281,2282,2283,2284,2285,2286,2290,2300,2310,2320,2330,2340,2350,2360,2370,2380,2381,2382,3002,3003,3004,3005,3007,3008,3010,3020,3030,3040,3050,3060,3080]
for numerical_part in saudi_tickers:
  ticker = str(numerical_part)
  ticker += '.'
  ticker += 'S'
  ticker += 'R'
  data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
  data["prv_low_1"] = data["Low"].shift(1)
  data["prv_low_2"] = data["Low"].shift(2)
  data["prv_low_3"] = data["Low"].shift(3)
  data["prv_low_4"] = data["Low"].shift(4)
  data["prv_low_5"] = data["Low"].shift(5)
  data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
  data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
  data['prv_high_1'] = data["High"].shift(1)
  data['prv_high_2'] = data["High"].shift(2)
  data['prv_high_3'] = data["High"].shift(3)
  data['prv_high_4'] = data["High"].shift(4)
  data['prv_high_5'] = data["High"].shift(5)
  data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
  data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
  data["seven_percent_of_close_price"] = .93*data["Close"]
  data["Close"] = 1*data["Close"]
  data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
  data["Last_Close"] = data["Close"].shift(1)
  data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
  data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
  data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
  data["buycumsum"] = data["buy"].cumsum().astype('int64')
  data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
  data["Close_up1%"] = 1.01*data["Close"]
  data["Close_down1%"] = .99*data["Close"]
  data.loc[(data.buy == False), 'Close_up1%'] = np.nan
  data.loc[(data.buy == False), 'Close_down1%'] = np.nan
  data["Close_up1%"] = data["Close_up1%"].ffill()
  data["Close_down1%"] = data["Close_down1%"].ffill()
  data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
  data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
  data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
  data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
  data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
  data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
  data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
  #data.tail(3)
  data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
  #data.head(3)
  data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
  #data.head(3)

  # exit and sell stock everytime a one percent loss is realized or exit and sell stock everytime a one percent gain is realized
  data.to_csv('out.csv', index=False)
  data = pd.read_csv('out.csv')
  data = data.drop_duplicates(subset="buycumsum",inplace=False)
  print("\n\n"+ticker)
  print("\n"+str(data["closeaboveup1%"].sum()))
  print("\n"+str(data["closebelowdown1%"].sum()))

[*********************100%***********************]  1 of 1 completed




1010.SR

13

68


[*********************100%***********************]  1 of 1 completed




1020.SR

12

97


[*********************100%***********************]  1 of 1 completed


...



3050.SR

10

58


[*********************100%***********************]  1 of 1 completed




3060.SR

17

80


[*********************100%***********************]  1 of 1 completed



3080.SR

10

53
